In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from stock_api import *
import numpy as np
import datetime as dt
from datetime import datetime,timedelta,time
import polars as pl
from mapping import *
from concurrent.futures import ThreadPoolExecutor, as_completed

# 统计涨停情况(分主板，创业板，科创板进行统计)
today = dt.date.today()
start_date = (today-timedelta(days=50)).strftime("%Y-%m-%d")
end_date = (today-timedelta(days=1)).strftime("%Y-%m-%d")
"""
计算涨停情况
:param df: DataFrame, 包含股票数据
:return: DataFrame, 添加了涨停情况和连板次数的DataFrame
"""
api = stock_api()


stock_list = get_all_stocks(date=end_date) #排除st,创业,科创,以及排除流通市值过大或过小
# 剔除创业板和科创板股票
stock_list = [stock for stock in stock_list if not (stock.split('.')[1].startswith('30') or stock.split('.')[1].startswith('688'))]
#stocks_data =get_history_symbols(symbols=stock_list, start_date=start_date, end_date=end_date)
print(f"股票池个数:{len(stock_list)}")
stocks_data = api.batch_get_history_symbols(stock_list,start_date=start_date, end_date=end_date)

# 统一列名
stocks_data['pct'] = (stocks_data['close']-stocks_data['pre_close'])/stocks_data['pre_close'] *100
stocks_data['vwap']= stocks_data['amount']/stocks_data['volume']


#stock_list = stock_list[0:1000]  # 只取前10只股票进行测试
print(f"数据日期:{stocks_data['trading_date'].unique().min()} - {stocks_data['trading_date'].unique().max()}")

将2488只股票分成13多线程批处理


股票池个数:2488


完成批次处理，股票代码: ['SHSE.600004', 'SHSE.600006', 'SHSE.600007']...(共200只)
完成批次处理，股票代码: ['SHSE.600345', 'SHSE.600348', 'SHSE.600350']...(共200只)
完成批次处理，股票代码: ['SHSE.600657', 'SHSE.600658', 'SHSE.600662']...(共200只)
完成批次处理，股票代码: ['SHSE.603006', 'SHSE.603008', 'SHSE.603009']...(共200只)
完成批次处理，股票代码: ['SHSE.600971', 'SHSE.600973', 'SHSE.600975']...(共200只)
完成批次处理，股票代码: ['SHSE.603353', 'SHSE.603355', 'SHSE.603357']...(共200只)
完成批次处理，股票代码: ['SZSE.000852', 'SZSE.000859', 'SZSE.000860']...(共200只)
完成批次处理，股票代码: ['SZSE.000301', 'SZSE.000338', 'SZSE.000400']...(共200只)
完成批次处理，股票代码: ['SZSE.002088', 'SZSE.002090', 'SZSE.002091']...(共200只)
完成批次处理，股票代码: ['SHSE.603889', 'SHSE.603890', 'SHSE.603893']...(共200只)
完成批次处理，股票代码: ['SZSE.002906', 'SZSE.002907', 'SZSE.002908']...(共88只)
完成批次处理，股票代码: ['SZSE.002331', 'SZSE.002332', 'SZSE.002333']...(共200只)
完成批次处理，股票代码: ['SZSE.002579', 'SZSE.002580', 'SZSE.002582']...(共200只)
所有批次处理完成，共合并81953条记录


数据日期:2025-11-17 - 2025-12-31


In [2]:

from mapping import *

def gm_add_auction(stock_data):
    """"
    利用掘金接口增加早盘数据current(symbols=stock_list,include_call_auction=True),主要是获取open即可
    分成pl和pd分别处理
    """
    # 将stock_data最后一天的股票代码
    if isinstance(stock_data, pl.DataFrame):
        stock_data = stock_data.sort(by=['trading_date', 'code']).reset_index(drop=True)
        last_date = stock_data.select(pl.col('trading_date').max()).item()
        stock_list = stock_data.filter(pl.col('trading_date') == last_date).select(pl.col('code')).to_series().to_list()

        new_data = current(symbols=stock_list,include_call_auction=True)
        new_data = pd.DataFrame(new_data)
        # 清洗数据
        new_data = clean_stocks_data(new_data)

        # 1. 将ts_data转为Polars
        new_data_pl = pl.from_pandas(new_data)
        new_data_pl = new_data_pl.with_columns(
            pl.col('trading_date').str.strptime(pl.Date, "%Y-%m-%d").alias('trading_date')
        )
        
        # 2. 统一所有列的数据类型（核心修复）
        # 先获取stock_data的完整schema
        target_schema = stock_data.schema
        
        # 逐个处理列：存在的列强制转换类型，不存在的列添加并设置类型
        for col, dtype in target_schema.items():
            if col in ts_data_pl.columns:
                # 强制转换已有列的类型为stock_data的类型
                ts_data_pl = ts_data_pl.with_columns(
                    pl.col(col).cast(dtype).alias(col)
                )
            else:
                # 添加缺失列并设置类型
                ts_data_pl = ts_data_pl.with_columns(
                    pl.lit(None, dtype=dtype).alias(col)
                )
        
        
        # 4. 严格按照stock_data的列顺序排序
        ts_data_pl = ts_data_pl.select(stock_data.columns)
        
        # 5. 合并
        concat_data = stock_data.vstack(ts_data_pl, in_place=False)

        # 5. 合并并重新排序（关键：确保时间顺序正确）
        concat_data = stock_data.vstack(new_data_pl, in_place=False)
        concat_data = concat_data.sort(by=['code', 'trading_date'])  # 按股票+日期排序

        # 6. 用前一交易日的close填充pre_close（核心修正）
        if 'pre_close' in concat_data.columns and 'close' in concat_data.columns:
            concat_data = concat_data.with_columns(
                pl.when(pl.col('pre_close').is_null())
                .then(pl.col('close').shift(1).over('code'))  # 取同一股票前一天的close
                .otherwise(pl.col('pre_close'))
                .alias('pre_close')
            )
        
    elif isinstance(stock_data, pd.DataFrame):
        # 1. 原始数据排序
        stock_data_sorted = stock_data.sort_values(by=['code', 'trading_date']).reset_index(drop=True)
        # 2. 取最后一个交易日
        last_date = stock_data_sorted['trading_date'].unique().max()
        # 3. 提取最后交易日的所有股票代码列表
        stock_list = stock_data_sorted[stock_data_sorted['trading_date'] == last_date]['code'].tolist()

        new_data = current(symbols=stock_list,include_call_auction=True)
        new_data = pd.DataFrame(new_data)
        new_data['trading_date'] = new_data['created_at']
        # 清洗数据
        new_data = clean_stocks_data(new_data)

        # 4. 获取需要给 ts_data 补充的列（stock_data 有而 ts_data 没有的列）
        # 使用 reindex 自动补齐并保留列顺序（pandas 会用 NaN/NaT 填充）
        new_data = new_data.reindex(columns=stock_data.columns)
        concat_data = pd.concat([stock_data, new_data], ignore_index=True)
        concat_data = concat_data.sort_values(by=['code', 'trading_date'])  # 按股票+日期排序
        
        # 5. 用前一交易日的close填充pre_close（核心修正）
        if 'pre_close' in concat_data.columns and 'close' in concat_data.columns:
            concat_data['pre_close'] = concat_data.groupby('code').apply(
                lambda group: group['pre_close'].fillna(group['close'].shift(1))
            ).reset_index(level=0, drop=True)  # 取同一股票前一天的close
    return concat_data


today_str = dt.date.today().strftime("%Y-%m-%d")
# 使用示例：
if stocks_data is not None:
    if today in stocks_data['trading_date'].unique():
        print(f"验证成功：数据中已包含 {today_str} 的行情")
    else:
        #stocks_data = ts_add_auction(stocks_data,m_ts)
        stocks_data = gm_add_auction(stocks_data)
        # 检查是否成功添加了今天的数据
        if today in stocks_data['trading_date'].unique():
            print(f"新增数据行数: {len(stocks_data[stocks_data['trading_date'] == today])}")
        else:
            print(f"添加失败未发现 {today_str} 的新增数据")
else:
    print("没有历史数据可添加最新行情")


C:\Users\20561\AppData\Local\Temp\ipykernel_48072\3015568918.py:79: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concat_data = pd.concat([stock_data, new_data], ignore_index=True)


新增数据行数: 2895


C:\Users\20561\AppData\Local\Temp\ipykernel_48072\3015568918.py:84: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  concat_data['pre_close'] = concat_data.groupby('code').apply(


In [3]:
# 需要的特征:涨停状态（未涨停,炸板,断板-三天之内有涨停），涨停描述(几天几板)，sma(均线)
import polars as pl
import pandas as pd
import numpy as np

def mark_limit_status(stock_data: pd.DataFrame) -> pd.DataFrame:
    """
    标记涨停状态（未涨停, 炸板, 断板, 正常涨停）
    """
    # 复制数据避免修改原数据
    stock_data = stock_data.copy()
    # 按股票代码和交易日排序
    stock_data = stock_data.sort_values(["code", "trading_date"])
    
    # 计算涨停和炸板标记
    stock_data["is_limit_up"] = (stock_data["close"] >= stock_data["limit_up"] * 0.999)
    stock_data["is_broken_limit"] = (
        (stock_data["high"] >= stock_data["limit_up"] * 0.999) & 
        (stock_data["close"] < stock_data["limit_up"] * 0.999)
    )
    
    # 断板标记：最近3天（不含当天）有涨停，且当天未涨停也未炸板
    def mark_db(group: pd.DataFrame) -> pd.DataFrame:
        is_limit_up = group["is_limit_up"].tolist()
        is_broken = group["is_broken_limit"].tolist()
        is_db = []
        limit_status_ext = []
        
        for i in range(len(is_limit_up)):
            # 取最近3天（不含当天）的数据
            recent_start = max(0, i - 2)
            recent = is_limit_up[recent_start:i]
            
            # 判断是否为断板
            if not is_limit_up[i] and not is_broken[i] and any(recent):
                is_db.append(True)
                limit_status_ext.append("断板")
            else:
                is_db.append(False)
                limit_status_ext.append(None)
        
        group["is_db"] = is_db
        group["limit_status_ext"] = limit_status_ext
        return group
    
    # 按股票代码分组处理
    stock_data = stock_data.groupby("code", group_keys=False).apply(mark_db)
    
    # 综合状态判断
    stock_data["limit_status"] = np.select(
        [
            stock_data["is_limit_up"],
            stock_data["is_broken_limit"],
            stock_data["is_db"]
        ],
        [
            "涨停",
            "炸板",
            "断板"
        ],
        default="未涨停"
    )
    
    return stock_data

def mark_limit_desc(stock_data: pd.DataFrame) -> pd.DataFrame:
    """
    标记涨停描述（几天几板），允许中间有断板，直到再次未涨停为止
    """
    stock_data = stock_data.copy()
    stock_data = stock_data.sort_values(["code", "trading_date"])
    
    def calc_desc(group: pd.DataFrame) -> pd.DataFrame:
        group = group.sort_values("trading_date").reset_index(drop=True)
        is_limit_up = group["is_limit_up"].tolist()
        limit_status = group["limit_status"].tolist()
        desc_list = []
        period_start = 0  # 当前周期起始索引
        
        for i in range(len(is_limit_up)):
            total_days = i - period_start + 1
            up_days = sum(is_limit_up[period_start:i+1])
            
            if limit_status[i] in ['涨停', '炸板']:
                desc_list.append(f"{total_days}天{up_days}板")
            elif limit_status[i] == '未涨停':
                desc_list.append("未涨停")
                period_start = i + 1  # 重置周期起点
            elif limit_status[i] == '断板':
                desc_list.append('断板')
            else:
                desc_list.append(None)
        
        group["limit_desc"] = desc_list
        return group
    
    return stock_data.groupby("code", group_keys=False).apply(calc_desc)


def mark_last_limit_desc(stock_data: pd.DataFrame) -> pd.DataFrame:
    """
    向前记录最后一次涨停状态的函数
    """
    stock_data = stock_data.copy()
    stock_data = stock_data.sort_values(["code", "trading_date"])
    
    def calc_last_desc(group: pd.DataFrame) -> pd.DataFrame:
        group = group.sort_values("trading_date").reset_index(drop=True)
        limit_status = group["limit_status"].tolist()
        limit_desc = group["limit_desc"].tolist()
        last_limit_desc_list = []
        period_start = 0
        last_valid_desc = None
        
        for i in range(len(limit_status)):
            if i == 0:
                last_limit_desc_list.append(None)
                continue
            
            pre_status = limit_status[i-1]
            
            if pre_status in ['涨停', '炸板', '断板']:
                # 向前搜索最近的涨停描述
                found = False
                for j in range(i-1, period_start-1, -1):
                    if limit_status[j] == '涨停':
                        last_valid_desc = limit_desc[j]
                        found = True
                        break
                last_limit_desc_list.append(last_valid_desc if found else None)
            else:
                last_limit_desc_list.append(None)
                period_start = i -1
                last_valid_desc = None
        
        group["last_limit_desc"] = last_limit_desc_list
        return group
    
    return stock_data.groupby("code", group_keys=False).apply(calc_last_desc)

def cal_n_lowest(stock_data: pd.DataFrame, window: int = 30, include_today: bool = False) -> pd.DataFrame:
    """
    计算股票n日内的最低股价
    
    参数:
        stock_data: 包含股票数据的DataFrame，需包含"code"（股票代码）、"trading_date"（交易日）、"low"（当日最低价）列
        window: 计算最低股价的窗口大小，默认30天（即“n日”中的n）
        include_today: 是否包含当天价格，默认为False（计算“前n天”最低值，不含当天；True则计算“包含当天在内的n天”最低值）
    
    返回:
        添加了n日内最低股价列的DataFrame，新增列名为 `lowest_{window}`（如window=30时为`lowest_30`）
    """
    # 1. 确保数据按“股票代码+交易日”排序
    stock_data = stock_data.sort_values(["code", "trading_date"]).copy()
    
    # 2. 按股票分组计算滚动最低价
    if include_today:
        # 包含当天：计算当前日及之前共window天的最低价
        rolling_min = stock_data.groupby("code")["low"].rolling(window=window, min_periods=1).min()
    else:
        # 不包含当天：先计算包含当天的窗口最小值，再整体后移1天
        rolling_min = stock_data.groupby("code")["low"].rolling(window=window, min_periods=1).min().shift(1)
    
    # 3. 添加结果列并返回
    col_name = f"lowest_{window}"
    stock_data[col_name] = rolling_min.reset_index(level=0, drop=True)
    
    return stock_data

# 计算量比
def add_volume_ratio(stock_data: pl.DataFrame, window: int = 5) -> pl.DataFrame:
    """
    计算量比（volume_ratio_window）
    量比 = 当前成交量 / 过去n天的平均成交量
    """
    return stock_data.with_columns([
        (pl.col("volume") / pl.col("volume").rolling_mean(window).over("code")).alias(f"volume_ratio_{window}")
    ])

# 计算k线特征
def cal_kline_pattern_features(stock_data: pl.DataFrame) -> pl.DataFrame:
    """
    body_ratio: 实体占比（实体长度/价格波动范围）
    upper_shadow_ratio: 上影线占比
    lower_shadow_ratio: 下影线占比
    candle_direction: 阴阳线标识（1=阳线，-1=阴线，0=十字星）
    计算k线形态特征
    """
    # 计算价格波动范围（仅内部使用，不保留为列）
    price_range = pl.col("high") - pl.col("low")
    
    return stock_data.with_columns([
        # 实体占比（实体长度/价格波动范围）
        ((pl.col("close") - pl.col("open")).abs() / price_range)
            .replace([float('inf'), -float('inf')], 0)
            .fill_null(0)
            .alias("body_ratio"),
        
        # 上影线占比
        (pl.when(pl.col("close") >= pl.col("open"))
            .then(pl.col("high") - pl.col("close"))  # 阳线
            .otherwise(pl.col("high") - pl.col("open"))  # 阴线
            / price_range)
            .replace([float('inf'), -float('inf')], 0)
            .fill_null(0)
            .alias("upper_shadow_ratio"),
        
        # 下影线占比
        (pl.when(pl.col("close") >= pl.col("open"))
            .then(pl.col("open") - pl.col("low"))  # 阳线
            .otherwise(pl.col("close") - pl.col("low"))  # 阴线
            / price_range)
            .replace([float('inf'), -float('inf')], 0)
            .fill_null(0)
            .alias("lower_shadow_ratio"),
        
        # 阴阳线标识（1=阳线，-1=阴线，0=十字星）
        pl.when(pl.col("close") > pl.col("open")).then(1)
            .when(pl.col("close") < pl.col("open")).then(-1)
            .otherwise(0)
            .alias("candle_direction")
    ])
    
def add_ema(stock_data: pl.DataFrame, window: int = 5) -> pl.DataFrame:
    """
    计算EMA（指数加权移动平均线），纯Polars实现
    参数:
        stock_data: Polars DataFrame，需包含'close'、'code'和'trading_date'列
        window: EMA窗口长度
    返回:
        添加了ema_{window}列的DataFrame
    """
    # 计算平滑系数alpha，与pandas的span参数保持一致
    alpha = 2 / (window + 1)
    
    # 按股票代码分组，按日期排序后计算EMA
    # 使用over('code')确保不同股票独立计算
    return stock_data.sort(['code', 'trading_date']).with_columns(
        pl.col('close')
        .ewm_mean(alpha=alpha, adjust=False)  # adjust=False与pandas保持一致
        .over('code')
        .alias(f'ema_{window}')
    )

def add_ewma_volatility(stock_data: pl.DataFrame, window: int = 10, alpha: float = 0.94) -> pl.DataFrame:
    """
    计算EWMA波动率（指数加权移动平均波动率）
    参数:
        stock_data: 需包含 "code"、"trading_date"、"pct" 列（pct为涨跌幅，如3表示3%）
        window: 初始方差的计算窗口（递归起点）
        alpha: 衰减系数（0<α≤1）
    返回:
        添加 ewma_volatility_{window} 列的DataFrame（单位：%，与pct列一致）
    """
    import numpy as np

    stock_data = stock_data.sort(["code", "trading_date"])

    # 步骤1：将 pct 转为小数形式（如3%→0.03），再计算平方收益率
    stock_data = stock_data.with_columns(
        (pl.col("pct") / 100).pow(2).alias("return_squared")
    )

    # 步骤2：计算初始方差（前window天的等权重方差，作为递归起点）
    stock_data = stock_data.with_columns(
        pl.col("return_squared")
        .rolling_var(window_size=window, min_periods=1, ddof=1)
        .over("code")
        .alias("initial_variance")
    )

    # 步骤3：分组递归计算EWMA方差和波动率
    def calc_ewma_vol(group: pl.DataFrame) -> pl.DataFrame:
        returns_sq = group["return_squared"].to_list()
        initial_var = group["initial_variance"].to_list()
        ewma_var = []
        for i in range(len(returns_sq)):
            if i == 0:
                v = initial_var[i]
                if v is None or (isinstance(v, float) and np.isnan(v)):
                    v = 0.0
                ewma_var.append(v)
            else:
                prev = ewma_var[i-1]
                if prev is None or (isinstance(prev, float) and np.isnan(prev)):
                    prev = 0.0
                ewma_var.append(alpha * returns_sq[i] + (1 - alpha) * prev)
        ewma_vol = [float(v ** 0.5) * 100 if v >= 0 else 0.0 for v in ewma_var]
        return group.with_columns([
            pl.Series(f"ewma_volatility_{window}", ewma_vol)
        ])

    stock_data = stock_data.group_by("code").map_groups(calc_ewma_vol)
    stock_data = stock_data.drop(["return_squared", "initial_variance"])
    return stock_data

def add_volume_concentration(stock_data: pl.DataFrame, window: int = 20, bins: int = 10) -> pl.DataFrame:
    """
    计算成交量密集区域因子：识别近N日成交量最密集的价格区间，因子值为该区间的中间价
    
    参数:
        stock_data: 包含股票数据的DataFrame，需包含"code"、"trading_date"、"close"、"volume"列
        window: 计算窗口大小，默认20天
        bins: 价格区间划分数量，默认10个区间
    
    返回:
        添加了volume_concentration_{window}列的DataFrame(volume_concentration_{window})
    """
    stock_data = stock_data.sort(["code", "trading_date"])
    
    def calc_concentration(group: pl.DataFrame) -> pl.DataFrame:
        # 获取窗口内的价格和成交量数据
        prices = group["vwap"].to_list()
        volumes = group["volume"].to_list()
        result = []
        
        for i in range(len(prices)):
            # 确定窗口范围
            start_idx = max(0, i - window + 1)
            window_prices = prices[start_idx:i+1]
            window_volumes = volumes[start_idx:i+1]
            
            if len(window_prices) < 2:  # 数据不足时返回NaN
                result.append(None)
                continue
            
            # 计算价格区间
            min_price = min(window_prices)
            max_price = max(window_prices)
            price_range = max_price - min_price
            
            if price_range < 1e-6:  # 价格无波动时
                result.append(min_price)
                continue
            
            # 划分价格区间并计算每个区间的成交量总和
            bin_volumes = [0.0] * bins
            for p, v in zip(window_prices, window_volumes):
                bin_idx = min(int((p - min_price) / price_range * bins), bins - 1)
                bin_volumes[bin_idx] += v
            
            # 找到成交量最大的区间
            max_bin_idx = bin_volumes.index(max(bin_volumes))
            
            # 计算该区间的中间价作为因子值
            bin_start = min_price + (max_bin_idx / bins) * price_range
            bin_end = min_price + ((max_bin_idx + 1) / bins) * price_range
            result.append((bin_start + bin_end) / 2)
        
        return group.with_columns([
            pl.Series(f"volume_concentration_{window}", result)
        ])
    
    return stock_data.group_by("code").map_groups(calc_concentration)

# 非流动性因子
def add_illiquidity(stock_data: pl.DataFrame, window: int = 20) -> pl.DataFrame:
    """
    计算非流动性因子（Amihud非流动性指标）
    衡量价格对成交量的敏感度，值越高表示流动性越差
    
    公式: 非流动性 = 平均(每日|收益率| / 成交额)，其中成交额=收盘价×成交量
    
    参数:
        stock_data: 包含股票数据的DataFrame，需包含"code"、"trading_date"、"close"、
                   "volume"、"pct"（涨跌幅百分比）列
        window: 计算窗口大小，默认20天
    
    返回:
        添加了illiquidity_{window}列的DataFrame
    """
    stock_data = stock_data.sort(["code", "trading_date"])
    
    # 计算每日成交额(收盘价×成交量)和收益率绝对值
    stock_data = stock_data.with_columns([
        pl.col("pct").abs().alias("abs_return")                 # 收益率绝对值
    ])
    
    # 计算每日非流动性指标（避免除以零）
    stock_data = stock_data.with_columns([
        pl.when(pl.col("turnover") > 0)
        .then(pl.col("abs_return") / pl.col("turnover")*10000) #单位w
        .otherwise(0)
        .alias("daily_illiquidity")
    ])
    
    # 计算滚动窗口平均值作为非流动性因子
    stock_data = stock_data.with_columns([
        pl.col("daily_illiquidity")
        .rolling_mean(window_size=window)
        .over("code")
        .alias(f"illiquidity_{window}")
    ])
    
    # 清理临时列
    return stock_data.drop(["turnover", "abs_return", "daily_illiquidity"])
"""
stock_data = add_illiquidity(stock_data,window=8)
stock_data = add_volume_concentration(stock_data,window=10,bins=10)
stock_data = stock_data.with_columns([
    pl.col("volume_concentration_10").shift(1).over("code").alias("pre_volume_concentration_10"),
])
"""

'\nstock_data = add_illiquidity(stock_data,window=8)\nstock_data = add_volume_concentration(stock_data,window=10,bins=10)\nstock_data = stock_data.with_columns([\n    pl.col("volume_concentration_10").shift(1).over("code").alias("pre_volume_concentration_10"),\n])\n'

In [4]:
# 1. 先确保数据按股票代码和日期排序
stocks_data = stocks_data.sort_values(['code', 'trading_date']).reset_index(drop=True)
# 标记涨停状态：limit_status
stocks_data = mark_limit_status(stocks_data)
# 标记涨停描述：limit_desc
stocks_data = mark_limit_desc(stocks_data)
# 记录最近的一次涨停描述：last_limit_desc
stocks_data = mark_last_limit_desc(stocks_data)
# 计算均线:sma_{window}
stocks_data['sma_7'] = sma(stocks_data['close'],window=7)
stocks_data['sma_10'] = sma(stocks_data['close'],window=10)
stocks_data = cal_n_lowest(stocks_data)
stocks_data['close_sma7_pct'] = (stocks_data['close']-stocks_data['sma_7'])/stocks_data['sma_7']*100
stocks_data['open_pct']=(stocks_data['open']-stocks_data['pre_close'])/stocks_data['pre_close']*100


# 2. 在每个股票组内计算移位数据（关键步骤）
# 按股票分组后进行移位操作
stocks_data['prev_limit_status'] = stocks_data.groupby('code')['limit_status'].shift(1)
stocks_data['prev_sma_7'] = stocks_data.groupby('code')['sma_7'].shift(1)
stocks_data['prev_sma_10'] = stocks_data.groupby('code')['sma_10'].shift(1)
stocks_data['pre_pct'] = stocks_data.groupby('code')['pct'].shift(1)
stocks_data['pre_vwap'] = stocks_data.groupby('code')['vwap'].shift(1)
stocks_data['pre_close_sma7_pct'] = stocks_data.groupby('code')['close_sma7_pct'].shift(1)

# 计算收盘价与7日均线的百分比差值
stocks_data['close_sma7_pct'] = (stocks_data['close'] - stocks_data['sma_7']) / stocks_data['sma_7'] * 100

# 3. 基于组内移位后的数据筛选买入信号
low = -5
high = -2.5
信号文件 = stocks_data[
    # 1. 昨日是断板或炸板
    (stocks_data["prev_limit_status"].isin(["断板", "炸板"])) &
    
    # 2. 今日低开幅度在-3%至-4%之间（根据您的描述调整了高低值）
    (stocks_data["open_pct"] >= low) & (stocks_data["open_pct"] <= high) &
    
    # 3. 昨日收盘在昨日5日均线上
    (stocks_data["pre_close"] >= stocks_data["prev_sma_7"]) &
    
    # 4. 最近一次涨停描述不是一天一板且不为空
    (
        #(stocks_data["last_limit_desc"] != "1天1板") & 
        (stocks_data["last_limit_desc"].notnull()) 
    ) 

    # 5. 自由流通值在30亿到1000亿之间（经过stock_list已经筛选）
    # &(
    #     (stocks_data["mv_A_free_float"] >= 30 * 1e8) &
    #     (stocks_data["mv_A_free_float"] <= 1000 * 1e8)
    # ) 
    
    # 6. 绝对位置不能太高，30日最低点至今涨幅不超过3倍
    &(stocks_data["open"] / stocks_data["lowest_30"] <= 3)
]

C:\Users\20561\AppData\Local\Temp\ipykernel_48072\447358059.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stock_data = stock_data.groupby("code", group_keys=False).apply(mark_db)
C:\Users\20561\AppData\Local\Temp\ipykernel_48072\447358059.py:97: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return stock_data.groupby("code", group_keys=False).apply(calc_desc)
C:\Users\20561\AppData\Local\Temp\ipykernel_48072\44

In [5]:
# 打印每日的股票代码
today_str = (dt.date.today()).strftime("%Y-%m-%d")
today_stocks = 信号文件[信号文件['trading_date']==today]
today_stocks.sort_values('close_sma7_pct')
filter_stocks = today_stocks[today_stocks['last_limit_desc']!='1天1板']
code_list = filter_stocks['code'].to_list()
# 提取每个code的后缀数字并打印
print('日期：{}'.format(today))
print('今日排除首板信号股票代码:')
for code in code_list:
    # 分割字符串，取小数点后的部分
    suffix_number = code.split('.')[1]
    print(suffix_number)

code_list = today_stocks['code'].to_list()
print('今日所有信号股票代码:')
for code in code_list:
    # 分割字符串，取小数点后的部分
    suffix_number = code.split('.')[1]
    print(suffix_number)
    

日期：2025-12-08
今日排除首板信号股票代码:
603327
000692
001211
002348
002632
今日所有信号股票代码:
603327
000692
001211
002348
002632


In [6]:
# ...existing code...
import pandas as pd
import polars as pl

def filter_recent_stocks(df, n: int = 3):
    """
    筛选最近n天的数据并返回(代码列表, 筛选后的同类型DataFrame)
    同时兼容 pandas 和 polars
    """
    # pandas 分支
    if isinstance(df, pd.DataFrame):
        df = df.copy()
        if not pd.api.types.is_datetime64_any_dtype(df["trading_date"]):
            df["trading_date"] = pd.to_datetime(df["trading_date"])
        # 统一到 date 粒度，避免时分秒干扰
        df["trading_date"] = df["trading_date"].dt.date

        trade_dates = sorted(df["trading_date"].unique().tolist(), reverse=True)
        recent_days = trade_dates[:min(n, len(trade_dates))]
        filtered_df = df[df["trading_date"].isin(recent_days)].copy()

        codes = (
            filtered_df["code"]
            .dropna()
            .astype(str)
            .tolist()
        )
        # 提取后缀并去重，保留出现顺序
        processed_codes = [c.split(".")[1] for c in codes if "." in c]
        processed_codes = list(dict.fromkeys(processed_codes))
        return processed_codes, filtered_df

    # polars 分支
    if isinstance(df, pl.DataFrame):
        # 统一 trading_date 为 pl.Date
        td_dtype = df.schema.get("trading_date")
        if td_dtype == pl.Datetime:
            df = df.with_columns(pl.col("trading_date").cast(pl.Date))
        elif td_dtype != pl.Date:
            df = df.with_columns(pl.col("trading_date").str.to_date())

        trade_dates = (
            df.select(pl.col("trading_date").unique())
              .to_series()
              .to_list()
        )
        trade_dates.sort(reverse=True)
        recent_days = trade_dates[:min(n, len(trade_dates))]

        filtered_df = df.filter(pl.col("trading_date").is_in(recent_days))

        codes = (
            filtered_df.select(pl.col("code"))
                       .drop_nulls()
                       .to_series()
                       .to_list()
        )
        processed_codes = [str(c).split(".")[1] for c in codes if isinstance(c, str) and "." in c]
        processed_codes = list(dict.fromkeys(processed_codes))
        return processed_codes, filtered_df

    raise TypeError("df 需为 pandas.DataFrame 或 polars.DataFrame")
# ...existing code...
code_list, today_stocks = filter_recent_stocks(信号文件, n=1)
today_stocks = today_stocks[today_stocks['last_limit_desc']!='1天1板']

# 获取最近3天的股票
code_list, recent_stocks = filter_recent_stocks(信号文件, n=3)
print('最近3日代码:')
stock_list = []
for index,stock in recent_stocks.iterrows():
    if stock['last_limit_desc'] != '1天1板':
        suffix_number = stock['code'].split('.')[1]
        if suffix_number not in stock_list:
            stock_list.append(suffix_number)
            print(suffix_number)



最近3日代码:
600828
600981
603068
603327
603778
000547
000592
000678
000692
001211
002084
002348
002413
002632
002702
002983


In [7]:
from stock_api import *
api = stock_api()
today_stocks['name'] = api.get_stock_name(today_stocks['code'])
today_stocks['pct'] = (today_stocks['close'] - today_stocks['pre_close'])/today_stocks['pre_close']*100
today_stocks['盈亏'] = ((today_stocks['close'] - today_stocks['open'])/today_stocks['open']*100)-0.3
# 今日股票复盘,汇报今日信号股票的收盘涨幅情况
win_rate = len(today_stocks[today_stocks['盈亏']>0])/len(today_stocks)*100
avg_profit = today_stocks['盈亏'].mean()
print('今日信号股票收盘涨幅情况:')
print(f"总共信号股票数:{len(today_stocks)},盈利股票数:{len(today_stocks[today_stocks['盈亏']>0])},胜率:{win_rate:.2f}%,平均盈亏:{avg_profit:.2f}%")
for index,stock in today_stocks.iterrows():
    suffix_number = stock['code'].split('.')[1]
    name = stock['name']
    当日盈亏 = stock['盈亏']
    
    print(f"{suffix_number}({name}): 涨跌幅{stock['pct']:.2f}%,当日盈亏:{当日盈亏:.2f}%")



今日信号股票收盘涨幅情况:
总共信号股票数:5,盈利股票数:5,胜率:100.00%,平均盈亏:5.86%
603327(福蓉科技): 涨跌幅8.16%,当日盈亏:11.91%
000692(惠天热电): 涨跌幅-3.10%,当日盈亏:0.94%
001211(双枪科技): 涨跌幅-3.34%,当日盈亏:1.41%
002348(高乐股份): 涨跌幅6.94%,当日盈亏:11.13%
002632(道明光学): 涨跌幅0.93%,当日盈亏:3.90%


In [8]:
# 近日窗口内股票复盘
